# Exploring the t5 model for text detoxification



In [ ]:
# installing huggingface libraries for dataset, models and metrics
!pip install datasets transformers[sentencepiece] sacrebleu

!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
# Necessary inputs
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

## Selecting the model
For the example purpose we select as model checkpoint the smallest transformer in T5 family - `t5_small`. Other pre-trained models can be found [here](https://huggingface.co/docs/transformers/model_doc/t5#:~:text=T5%20comes%20in%20different%20sizes%3A).

In [ ]:
# selecting model checkpoint
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the WMT16 dataset
#raw_datasets = load_dataset("wmt16", "de-en")

# Load the BLUE metric
metric = load_metric("sacrebleu")

In [ ]:
df = pd.read_csv("processed.tsv", sep ="\t")
df.head(5)

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,I've spent the last seven years with four smel...,I've spent the last seven years with four men.,0.950000,0.129630,0.990042,0.000041
1,tell me something I don't fucking know.,Tell me something I don't know!,0.949999,0.200000,0.971418,0.000079
2,you're such a fucking tough guy.,You're such a Tough guy.,0.949997,0.242424,0.977852,0.000238
3,someone tried to kill her with an injection of...,Someone tried to kill her by injecting the virus?,0.949996,0.137931,0.980873,0.102180
4,I could look at women's legs for hours.,I could look at a woman's legs for hours.,0.949996,0.047619,0.941838,0.006438


## Dataset
Downloaded from HuggingFace dataset is a `DatasetDict`. It contains keys `["train", "validation", "test"]` - which represents a dataset splits

In [ ]:
#df.drop(columns =['lenght_diff', 'similarity', 'ref_tox', 'trn_tox'])

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [ ]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
validation_dataset = datasets.Dataset.from_dict(validate)

In [ ]:
raw_datasets = datasets.DatasetDict({"train":train_dataset,"test":test_dataset, "validation": validation_dataset})

In [ ]:
# samples from train dataset
raw_datasets["train"][:5]

{'reference': ["Every woman in California thinks... she's either too fat or too thin or too something.",
  "Listen, give me your dog, and I'll forget it. You're nuts.",
  'Without my rifle, I am useless.',
  'You guys are worse than NASA.',
  "I'm sorry I can't crap like Bono."],
 'translation': ["every woman in California thinks she's too fat or too skinny.",
  "listen, give me your dog, and I'll forget it.",
  '"I\'m useless without my rifle.',
  "you're worse than NASA.",
  "I'm sorry I don't like Bono."],
 'similarity': [0.91169195304,
  0.937651404656,
  0.949683975862,
  0.933995157371,
  0.906056473784],
 'lenght_diff': [0.2873563218390804,
  0.2203389830508474,
  0.03125,
  0.2,
  0.1470588235294117],
 'ref_tox': [0.9432735443115234,
  0.999486804008484,
  0.9850885272026062,
  0.8082457184791565,
  0.9785056114196776],
 'trn_tox': [0.4230548143386841,
  0.0024958848953247,
  0.2508155405521393,
  0.090722881257534,
  0.0002564493624959]}

In [ ]:
fake_preds = ["hello there", "general kenobi", "Can I get an A"]
fake_labels = [["hello there"], ["general kenobi"], ['Can I get a C']]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 45.59274666224604,
 'counts': [7, 4, 1, 0],
 'totals': [9, 6, 3, 2],
 'precisions': [77.77777777777777,
  66.66666666666667,
  33.333333333333336,
  25.0],
 'bp': 1.0,
 'sys_len': 9,
 'ref_len': 9}

## Preprocessing the data
As usual we will need to preprocess data and tokenize it before passing to model

In [ ]:
from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
# prefix for model input
prefix = "Detoxify this sentence:"

In [ ]:
raw_datasets["train"][:5]

{'reference': ["Every woman in California thinks... she's either too fat or too thin or too something.",
  "Listen, give me your dog, and I'll forget it. You're nuts.",
  'Without my rifle, I am useless.',
  'You guys are worse than NASA.',
  "I'm sorry I can't crap like Bono."],
 'translation': ["every woman in California thinks she's too fat or too skinny.",
  "listen, give me your dog, and I'll forget it.",
  '"I\'m useless without my rifle.',
  "you're worse than NASA.",
  "I'm sorry I don't like Bono."],
 'similarity': [0.91169195304,
  0.937651404656,
  0.949683975862,
  0.933995157371,
  0.906056473784],
 'lenght_diff': [0.2873563218390804,
  0.2203389830508474,
  0.03125,
  0.2,
  0.1470588235294117],
 'ref_tox': [0.9432735443115234,
  0.999486804008484,
  0.9850885272026062,
  0.8082457184791565,
  0.9785056114196776],
 'trn_tox': [0.4230548143386841,
  0.0024958848953247,
  0.2508155405521393,
  0.090722881257534,
  0.0002564493624959]}

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "toxic"
target_lang = "detoxified"

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# example of preprocessing
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[374, 235, 226, 4921, 48, 7142, 10, 18403, 2335, 16, 1826, 317, 7, 233, 255, 31, 7, 893, 396, 2886, 42, 396, 5551, 42, 396, 424, 5, 1], [374, 235, 226, 4921, 48, 7142, 10, 21310, 35, 6, 428, 140, 39, 1782, 6, 11, 27, 31, 195, 2612, 34, 5, 148, 31, 60, 11446, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[334, 2335, 16, 1826, 317, 7, 255, 31, 7, 396, 2886, 42, 396, 28451, 5, 1], [3011, 6, 428, 140, 39, 1782, 6, 11, 27, 31, 195, 2612, 34, 5, 1]]}

In [ ]:
# for the example purpose we will crop the dataset and select first 5000 for train
# and 500 for validation and test
cropped_datasets = raw_datasets
tokenized_datasets = cropped_datasets.map(preprocess_function, batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'reference': "Every woman in California thinks... she's either too fat or too thin or too something.",
 'translation': "every woman in California thinks she's too fat or too skinny.",
 'similarity': 0.91169195304,
 'lenght_diff': 0.2873563218390804,
 'ref_tox': 0.9432735443115234,
 'trn_tox': 0.4230548143386841,
 'input_ids': [374,
  235,
  226,
  4921,
  48,
  7142,
  10,
  18403,
  2335,
  16,
  1826,
  317,
  7,
  233,
  255,
  31,
  7,
  893,
  396,
  2886,
  42,
  396,
  5551,
  42,
  396,
  424,
  5,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [334,
  2335,
  16,
  1826,
  317,
  7,
  255,
  31,
  7,
  396,
  2886,
  42,
  396,
  28451,
  5,
  1]}

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# defining the parameters for training
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-detoxify",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
)

In [ ]:
# instead of writing collate_fn function we will use DataCollatorForSeq2Seq
# simliarly it implements the batch creation for training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.613400,1.242498,39.568700,11.808400
2,1.323100,1.173971,40.935400,11.799200
3,1.263900,1.144733,41.311700,11.762000
4,1.227400,1.125860,41.639100,11.772400
5,1.203600,1.114355,41.806500,11.765500
6,1.181200,1.105925,41.968200,11.769300
7,1.174600,1.099475,42.046900,11.762100
8,1.165800,1.095707,42.146900,11.770600
9,1.163100,1.093117,42.135100,11.762100
10,1.156600,1.092396,42.124800,11.759400


TrainOutput(global_step=9380, training_loss=1.2360559003947895, metrics={'train_runtime': 2134.6969, 'train_samples_per_second': 140.535, 'train_steps_per_second': 4.394, 'total_flos': 4069914065240064.0, 'train_loss': 1.2360559003947895, 'epoch': 10.0})

In [ ]:
# saving model
trainer.save_model('best')

In [ ]:
# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [ ]:
def translate(model, inference_request, tokenizer=tokenizer):
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0))

In [43]:
inference_request = prefix + 'You are a fucking piece of shit'
translate(model, inference_request,tokenizer)

you're a piece of shit.


In [45]:
inference_request = prefix + "kill yourself"
translate(model, inference_request,tokenizer)

Kill yourself


This is just because I'm using a google collab

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!zip -r /content/best_t5.zip /content/best

  adding: content/best/ (stored 0%)
  adding: content/best/generation_config.json (deflated 29%)
  adding: content/best/pytorch_model.bin (deflated 9%)
  adding: content/best/tokenizer.json (deflated 74%)
  adding: content/best/tokenizer_config.json (deflated 95%)
  adding: content/best/config.json (deflated 62%)
  adding: content/best/special_tokens_map.json (deflated 86%)
  adding: content/best/spiece.model (deflated 48%)
  adding: content/best/training_args.bin (deflated 50%)
